<a href="https://colab.research.google.com/github/pycriador/PyMalaDireta/blob/main/%5BUpload%5D_PyMalaDireta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Projeto de Fevereiro

Um programa simples para criar seus arquivos de mala direta utilizando um template em XLSX (Excel) e um documento DOCX (Word).

**Primeiro**, vamos instalar as dependências do Python? Clique no Play e
aguarde.

In [71]:
!pip install docxtpl pandas numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Essa classe é a que vai fazer todo o trabalho, você poderá customizar alguns campos se desejar, por exemplo, a classe pega do arquivo XLSX uma coluna chamada "Nome" e "CPF", se você não tem esses campos na sua planilha, você poderá mudar o nome do arquivo final.

In [72]:
from docxtpl import DocxTemplate
import pandas as pd
import numpy as np
import time
import subprocess

class appMalaDireta():

    def read_template(self, templates='templates', page=None, encoding='ISO-8859-1'):
      print('Lendo o template')
      print(f"{templates}//{page}")
      self.template = DocxTemplate(f"{templates}//{page}")

    def read_xlsx(self, file=None, sheet=None):
      xlsx = pd.ExcelFile(file)
      return pd.read_excel(xlsx)

    def create(self, page=None, export_name=None, templates_path=None, encoding=None, 
               data=pd.DataFrame(), export_docx=None):

      for index, row in data.iterrows():
          timestamp = int(time.time())
          #Ler HTML como template
          self.read_template(templates=templates_path, page=page, encoding=encoding)
          #### Você poderá mudar o nome do arquivo final aqui
          #### Coloque entre {row['COLUNA']} o nome da coluna do seu Excel
          #### Se você não sabe o que fazer nessa parte, e seu arquivo não tem
          #### uma coluna NOME e CPF, só #comente as duas próximas linhas
          #CPF = ''.join(filter(str.isdigit, row["CPF"]))
          export_name = f'{row["NOME"].upper().strip()}-{row["CNPJ"]}'
          #export_name = 'X'
          #print(f'Gerando o arquivo: {export_name}')
          #Converter Nan em nada
          dados = row.replace(np.nan, '', regex=True)
          #Converter dados em string
          dados = dados.apply(str)
          self.template.render(dados.str.upper())
          print(f'Gerando o DOCX: {export_docx}/{export_name}-{timestamp}.docx')
          self.template.save(f'{export_docx}/{export_name}-{timestamp}.docx')

Agora vamos apagar os arquivos desnecessários? Se você já executou pelo menos uma vez todo esse Notebook, então vamos ter arquivos do upload anterior. É só executar o próximo passo que eu já deleto todos os arquivos para você.

In [ ]:
import os
import shutil
import glob

print('Limpando arquivos antigos XLSX')
fileList = glob.glob('*.xlsx', recursive=True)

for filePath in fileList:
    try:
        os.remove(filePath)
    except:
        print("Erro ao remover:", filePath)

print('Limpando arquivos antigos DOCX')
fileList = glob.glob('*.docx', recursive=True)

for filePath in fileList:
    try:
        os.remove(filePath)
    except:
        print("Erro ao remover:", filePath)

print('Limpando arquivos antigos')
try:
  shutil.rmtree('arquivos')
except Exception as e:
  print(f'Erro: {e}')

print('Limpando arquivos antigos')
try:
  shutil.rmtree('pdf')
except Exception as e:
  print(f'Erro: {e}')

print('Limpando templates')
try:
  shutil.rmtree('template')
except Exception as e:
  print(f'Erro: {e}')

print('Limapando arquivos de exemplo')
try:
  shutil.rmtree('sample_data')
except Exception as e:
  print(f'Erro: {e}')

print('Criando pasta para os arquivos')
try:
  os.mkdir('arquivos')
except Exception as e:
  print(f'Erro: {e}')

print('Criando pasta para os templates')
try:
  os.mkdir('template')
except Exception as e:
  print(f'Erro: {e}')

Agora chegou a hora de escolher os arquivos para mala direta. Primeiro, você escolherá o arquivo com a base de nomes, no formato XLSX.

In [ ]:
from google.colab import files

print('Escolha o arquivo com os dados em XLSX')
ap_up = files.upload()
app = appMalaDireta()

for ap in ap_up.keys():
  base = app.read_xlsx(file=ap)

Agora, você escolherá o template em DOCX para a carta. Nessa parte, o próprio programa vai preencher o template com as informações da base em Excel e gerar um DOCX para cada registro.
Não se esqueça, nesse arquivo os campos que você quer substituir com os valores do arquivo em Excel devem estar assim {{NOME}}.

Ex. Na planilha está com a coluna NOME, no template onde deve aparecer o nome deve estar assim: {{NOME}}


In [ ]:
from google.colab import files

print('Escolha o arquivo com o template em DOCX')
td_up = files.upload()

for td_up in td_up.keys():
  shutil.move(td_up, 'template')
  app.create(export_name='', templates_path='template', page=td_up, encoding='ISO-8859-1', data=base, export_docx='arquivos')

print('Abrir o diretório "arquivos" e fazer download dos arquivos')

Se você chegou aqui e não deu nenhum erro, então está tudo OK com os seus arquivos, criei uma pasta chamada **arquivos**, com todos os seus documentos em DOCX.

In [ ]:
import os.path
import os

if os.path.isfile('LibreOffice-7.2.5-x86_64.AppImage'):
  print('LibreOffice Portátil já está no seu Google Drive')
  !chmod +x LibreOffice-7.2.5-x86_64.AppImage
else:
  !wget https://libreoffice.soluzioniopen.com/stable/basic/LibreOffice-7.2.5-x86_64.AppImage
  !chmod +x LibreOffice-7.2.5-x86_64.AppImage

print('Limapando pasta de PDF')
try:
  shutil.rmtree('pdf')
except Exception as e:
  print(f'Erro: {e}')

!mkdir pdf

!./LibreOffice-7.2.5-x86_64.AppImage --headless --norestore --writer --convert-to pdf --outdir /content/pdf /content/arquivos/*.docx